In [1]:
# import libraries
import category_encoders as ce
import pandas as pd
import glob
import json
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler



In [2]:
# use list comprehention and glob to create one db from many
# works because he each db has the same columns
df1 = pd.concat([pd.read_csv(x) for x in glob.glob('data/Kickstarter*.csv')], ignore_index = True)


In [3]:
pd.options.display.max_columns=50
df1.head(2)

,backers_count,blurb,category,converted_pledged_amount,country,country_displayable_name,created_at,creator,currency,currency_symbol,currency_trailing_code,current_currency,deadline,disable_communication,friends,fx_rate,goal,id,is_backing,is_starrable,is_starred,launched_at,location,name,permissions,photo,pledged,profile,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,61,Support great art! Join us as we re-stage the ...,"{""id"":254,""name"":""Performances"",""slug"":""dance/...",4618,US,the United States,1579292017,"{""id"":2092817311,""name"":""Brooklyn Ballet"",""slu...",USD,$,True,USD,1583025192,False,NaN,1.0,3000.0,1568973024,NaN,False,NaN,1580433192,"{""id"":12589335,""name"":""Brooklyn"",""slug"":""brook...",Revisionist History 2,NaN,"{""key"":""assets/027/785/229/436e3e732fb36a494fe...",4618.0,"{""id"":3914215,""project_id"":3914215,""state"":""in...",revisionist-history-2,https://www.kickstarter.com/discover/categorie...,True,True,successful,1583025192,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",4618.0,domestic
1,52,JinBucha is a new kind of Brewery in North Par...,"{""id"":307,""name"":""Drinks"",""slug"":""food/drinks""...",3461,US,the United States,1446051515,"{""id"":1468694331,""name"":""Jing Chen"",""slug"":""ji...",USD,$,True,USD,1450118057,False,NaN,1.0,20000.0,1782378629,NaN,False,NaN,1447526057,"{""id"":2487889,""name"":""San Diego"",""slug"":""san-d...",JinBucha: A Modern Kombucha tasting room in No...,NaN,"{""key"":""assets/012/291/675/7fb6348cc8cb988e243...",3461.0,"{""id"":2198010,""project_id"":2198010,""state"":""in...",jinbucha-a-modern-kombucha-tasting-room-in-nor...,https://www.kickstarter.com/discover/categorie...,False,True,failed,1450118057,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",3461.0,domestic


In [4]:
df1.columns

Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'country_displayable_name', 'created_at', 'creator',
       'currency', 'currency_symbol', 'currency_trailing_code',
       'current_currency', 'deadline', 'disable_communication', 'friends',
       'fx_rate', 'goal', 'id', 'is_backing', 'is_starrable', 'is_starred',
       'launched_at', 'location', 'name', 'permissions', 'photo', 'pledged',
       'profile', 'slug', 'source_url', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type'],
      dtype='object')

In [5]:
df1.rename(columns={'category':'category_string', 'country':'country_id', 'country_displayable_name':'country', 'state':'campaign_success'}, inplace=True)

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219688 entries, 0 to 219687
Data columns (total 38 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   backers_count             219688 non-null  int64  
 1   blurb                     219680 non-null  object 
 2   category_string           219688 non-null  object 
 3   converted_pledged_amount  219688 non-null  int64  
 4   country_id                219688 non-null  object 
 5   country                   219688 non-null  object 
 6   created_at                219688 non-null  int64  
 7   creator                   219688 non-null  object 
 8   currency                  219688 non-null  object 
 9   currency_symbol           219688 non-null  object 
 10  currency_trailing_code    219688 non-null  bool   
 11  current_currency          219688 non-null  object 
 12  deadline                  219688 non-null  int64  
 13  disable_communication     219688 non-null  b

In [42]:
df = df1.copy()

In [43]:
df['blurb'] = df['blurb'].astype(str)
df['blurb_length']= df['blurb'].apply(lambda x: len(x))
df['blurb_length'].describe()


count    219688.000000
mean        107.435550
std          30.364783
min           1.000000
25%          91.000000
50%         120.000000
75%         131.000000
max         200.000000
Name: blurb_length, dtype: float64

In [44]:
df['blurb_length'].value_counts().sort_index()
# df['blurb_length'].value_counts()

1       29
2        8
3       19
4       20
5       17
      ... 
150    160
151      4
152      1
153      1
200      1
Name: blurb_length, Length: 154, dtype: int64

## Prepare Category & Subcategory Data for Back End

In [45]:
#prepare category string to be split into several columns
df['category_string']= df['category_string'].map(eval)

In [46]:
# split category into several columns
df_c= df['category_string'].apply(pd.Series)
df_c = df_c.drop(['id', 'position', 'parent_id', 'color', 'urls'], axis=1)
df_c.rename(columns={'name':'subcategory', 'slug':'cat_subcat', 'parent_name':'category'}, inplace=True)
df_c['category'].fillna(df_c['cat_subcat'], inplace=True)
df_c['category'] = df_c['category'].str.capitalize()
df['category']=df_c['category']
df['subcategory']=df_c['subcategory']
df['cat_subcat']=df_c['cat_subcat']


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219688 entries, 0 to 219687
Data columns (total 42 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   backers_count             219688 non-null  int64  
 1   blurb                     219688 non-null  object 
 2   category_string           219688 non-null  object 
 3   converted_pledged_amount  219688 non-null  int64  
 4   country_id                219688 non-null  object 
 5   country                   219688 non-null  object 
 6   created_at                219688 non-null  int64  
 7   creator                   219688 non-null  object 
 8   currency                  219688 non-null  object 
 9   currency_symbol           219688 non-null  object 
 10  currency_trailing_code    219688 non-null  bool   
 11  current_currency          219688 non-null  object 
 12  deadline                  219688 non-null  int64  
 13  disable_communication     219688 non-null  b

In [48]:
# drop duplicate subcategories
df_c = df_c.drop_duplicates(subset=['cat_subcat'])
df_c = df_c.drop(columns=['cat_subcat'])
df_c = df_c.sort_values(by=['category', 'subcategory'])
columns_names = ['category', 'subcategory']
df_c = df_c.reindex(columns=columns_names)
df_c.head(10)

,category,subcategory
1275,Art,Art
519,Art,Ceramics
45,Art,Conceptual Art
1372,Art,Digital Art
435,Art,Illustration
5384,Art,Installations
1794,Art,Mixed Media
13,Art,Painting
3789,Art,Performance Art
434,Art,Public Art


In [49]:
# drop duplicates contries
df_country = df.copy()
df_country = df_country[['country']]
df_country = df_country.sort_values(by=['country'])
df_country = df_country.drop_duplicates()
df_country

,country
167670,Australia
153397,Austria
175552,Belgium
98606,Canada
2301,Denmark
161308,France
86841,Germany
10371,Hong Kong
149854,Ireland
69391,Italy


In [50]:
df_c.to_json(r'C:\Users\whats\DS_Unit_3\Kickstart\categories_subcategories.json')
df_country.to_json(r'C:\Users\whats\DS_Unit_3\Kickstart\country.json')


## Row Evaluation

In [51]:
# check for duplicates of individual projects
dup_id = len(df[df.duplicated(subset='id')])
print(f"There are {dup_id} ids listed more than once.")

There are 27669 ids listed more than once.


In [52]:
df_dup_rows = df[df.duplicated(subset='id', keep=False)]
df_dup_rows.shape

(55336, 42)

In [53]:
# discover how many rows contain duplicates
dup_rows = len(df_dup_rows)
print(f"There are {dup_rows} rows containing duplicates.")
print(f"That is an average of {(dup_rows/dup_id):.2f} copies per duplicated item.")

There are 55336 rows containing duplicates.
That is an average of 2.00 copies per duplicated item.


In [54]:
# check state values
# combine canceled with failed?  & drop live?
# make booklean
print(df['campaign_success'].value_counts())
print("---"*15)

successful    128156
failed         76137
canceled        9138
live            6257
Name: campaign_success, dtype: int64
---------------------------------------------


## Drop rows

In [55]:
print(df.shape)
def row_wrangle(X):
    X = X.copy()
    # drop campaigns which are live or canceled
    # drop duplicate id rows
    X = X[X.campaign_success !='canceled']
    X = X[X.campaign_success != 'live']
    X = X.drop_duplicates(subset=['id'])
    X = X[X.blurb_length > 1]
    X = X.reset_index(drop=True)
    return(X)
df = row_wrangle(df)
print(df.shape)

(219688, 42)
(179456, 42)


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179456 entries, 0 to 179455
Data columns (total 42 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   backers_count             179456 non-null  int64  
 1   blurb                     179456 non-null  object 
 2   category_string           179456 non-null  object 
 3   converted_pledged_amount  179456 non-null  int64  
 4   country_id                179456 non-null  object 
 5   country                   179456 non-null  object 
 6   created_at                179456 non-null  int64  
 7   creator                   179456 non-null  object 
 8   currency                  179456 non-null  object 
 9   currency_symbol           179456 non-null  object 
 10  currency_trailing_code    179456 non-null  bool   
 11  current_currency          179456 non-null  object 
 12  deadline                  179456 non-null  int64  
 13  disable_communication     179456 non-null  b

In [57]:
# choose to ignore non USD values
df['current_currency'].value_counts()

USD    179407
AUD        41
EUR         8
Name: current_currency, dtype: int64

In [58]:
df['blurb_length'].value_counts().sort_index()

2        2
3        6
4        7
5       11
6       13
      ... 
149     82
150    155
151      4
153      1
200      1
Name: blurb_length, Length: 152, dtype: int64

In [59]:
df.describe()

,backers_count,converted_pledged_amount,created_at,deadline,fx_rate,goal,id,launched_at,pledged,state_changed_at,static_usd_rate,usd_pledged,blurb_length
count,179456.000000,1.794560e+05,1.794560e+05,1.794560e+05,179456.000000,1.794560e+05,1.794560e+05,1.794560e+05,1.794560e+05,1.794560e+05,179456.000000,1.794560e+05,179456.000000
mean,148.874978,1.336706e+04,1.468806e+09,1.475791e+09,0.992772,5.065833e+04,1.073176e+09,1.472947e+09,2.554876e+04,1.475791e+09,1.003930,1.336414e+04,109.124142
std,960.843985,1.048699e+05,7.427843e+07,7.401401e+07,0.216976,1.198595e+06,6.188916e+08,7.402606e+07,1.365928e+06,7.401404e+07,0.236859,1.047623e+05,29.293015
min,0.000000,0.000000e+00,1.240366e+09,1.242468e+09,0.009499,1.000000e-02,1.852000e+04,1.240674e+09,0.000000e+00,1.242468e+09,0.008771,0.000000e+00,2.000000
25%,4.000000,1.110000e+02,1.416500e+09,1.424900e+09,1.000000,1.500000e+03,5.368545e+08,1.421997e+09,1.170000e+02,1.424900e+09,1.000000,1.119307e+02,94.000000
50%,27.000000,1.564000e+03,1.466001e+09,1.473308e+09,1.000000,5.000000e+03,1.073278e+09,1.470489e+09,1.600000e+03,1.473308e+09,1.000000,1.565701e+03,122.000000
75%,90.000000,6.564250e+03,1.530118e+09,1.537138e+09,1.000000,1.500000e+04,1.608160e+09,1.534352e+09,7.013000e+03,1.537138e+09,1.000000,6.567250e+03,132.000000
max,105857.000000,1.296961e+07,1.599594e+09,1.600315e+09,1.777829,1.000000e+08,2.147476e+09,1.599797e+09,4.816218e+08,1.600315e+09,1.716408,1.296961e+07,200.000000


In [60]:
print(df['disable_communication'].value_counts())
print("---"*15)
print('')
print(df['is_starrable'].value_counts())
print("---"*15)
print('')
print(df['category'].nunique(), 'category items')

False    179456
Name: disable_communication, dtype: int64
---------------------------------------------

False    179456
Name: is_starrable, dtype: int64
---------------------------------------------

15 category items


## Column Evaluation

In [61]:
# columns to retain as-is (6)
keepers = ['country', 'goal', 'blurb','category', 'launched_at', 'sub_category']

# campaign_success, target column (1)

In [62]:
# future data columns, includes 4 mostly null columns(17)
'''
'spotlight' only applies to successfully funded projects,
'is_starrable' is only true with live campaigns,
'''

future_data = ['backers_count', 'converted_pledged_amount', 'created_at', 'friends',
                'id', 'is_backing', 'is_starred', 'permissions', 'photo',
                'pledged', 'profile', 'source_url', 'spotlight', 'staff_pick',
                'state_changed_at', 'urls', 'usd_pledged']

In [63]:
# extraneous columns (11)
'''
'disable_communications' only contains the value 'false',
'is_starrable' only contains the value 'false',
'country_displayable_name' duplicate information,

'''
extra_columns = ['category_string', 'country_id','creator',
                'disable_communication', 'deadline', 'is_starrable', 'location',
                'name', 'slug', 'cat_subcat', 'blurb_length']


In [64]:
# currency (7)
#'fx_rate' is the foriegn exchange rate,
currency_columns = ['currency', 'currency_symbol', 'currency_trailing_code',
                'current_currency', 'fx_rate', 'static_usd_rate', 'usd_type']

In [65]:
len(df)

179456

## Wrangle Data

In [66]:
def wrangle(X):
    # copy to avoid errors
    X = X.copy()

    # make boolean success column
    X['campaign_success'] = X['campaign_success'].replace({'failed':0, 'successful':1})

    # times are provided in UNIX in seconds
    # subtraction & divided by will provide length in days
    X['campaign_length'] = ((X['deadline']-X['launched_at'])/(60*24*60))

    # Drop future, extraneous & duplicate columns
    X = X.drop(columns=future_data)
    X = X.drop(columns=extra_columns)

    # Currency columns
    X = X.drop(columns=currency_columns)

    return(X)

kickstart = wrangle(df)


In [67]:
kickstart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179456 entries, 0 to 179455
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   blurb             179456 non-null  object 
 1   country           179456 non-null  object 
 2   goal              179456 non-null  float64
 3   launched_at       179456 non-null  int64  
 4   campaign_success  179456 non-null  int64  
 5   category          179456 non-null  object 
 6   subcategory       179456 non-null  object 
 7   campaign_length   179456 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 11.0+ MB


In [68]:
kickstart.to_csv(r'C:\Users\whats\DS_Unit_3\Kickstart\clean_kickstart_data.csv', index = False)
